In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchvision.transforms import transforms, ToTensor
from torchvision.datasets import ImageFolder
from tqdm.notebook import tqdm #DataLoader로 돌리기 때문에 tqdm 사용 가능

In [2]:
device=('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size=20

train_transforms = transforms.Compose(
    [
        transforms.CenterCrop(224),
        transforms.ToTensor(), # 이미지를 텐서로
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #RGB 채널별로 mean, std -- RGB 0~1사이의 값으로 바꾸고 나서의 mean
    ]
)

In [4]:
test_transforms = transforms.Compose(
    [
        transforms.Resize(255), # 이미지 리사이즈
        transforms.CenterCrop(224), # 중앙 224 x 244 크롭
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

In [5]:
# training, validation, testset 로드
trainset = ImageFolder(root='./data/dogs_vs_cats/train', transform=train_transforms)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                          drop_last=True)
                                                              
# 테스트 셋은 셔플 안함
testset = ImageFolder(root='./data/dogs_vs_cats/test', transform=test_transforms)
testloader = DataLoader(testset, batch_size=batch_size, drop_last=True)

print(len(trainset))
print(len(testset))

20000
5000


In [6]:
print(trainset.classes)
print(trainset.class_to_idx)
print(testset.classes)
print(testset.class_to_idx)

['cats', 'dogs']
{'cats': 0, 'dogs': 1}
['cats', 'dogs']
{'cats': 0, 'dogs': 1}


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # input image = 224 x 244 x 3 #이미지* 4

        # 224 x 224 x 3 --> 112 x 112 x 32 maxpool
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) #입력, 출력, 커널사이즈 (3*3), padding=1 (위아래를 0으로 채워진 1px로 준 후 3*3커널 통과하면 사이즈 안변하니까)
        # 112 x 112x 32 --> 56 x 56 x 64 maxpool
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) 
        # 56 x 56 x 64 --> 28 x 28 x 128 maxpool
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)     
        # 28 x 28 x 128 --> 14 x 14 x 128 maxpool
        self.conv4 = nn.Conv2d(128, 128, 3, padding=1)    

        # maxpool 2 x 2
        self.pool = nn.MaxPool2d(2, 2)

        # 28 x 28 x 128 vector flat 512개
        self.fc1 = nn.Linear(128 * 14 * 14, 512) #출력 512
        # 카테고리 2개 클래스
        self.fc2 = nn.Linear(512, 2) 

        # dropout 적용
        self.dropout = nn.Dropout(0.5) # 0.25 해보고 0.5로 해보기. 값 저장하고나서

    def forward(self, x):
        # conv1 레이어에 relu 후 maxpool. 112 x 112 x 32
        x = self.pool(torch.relu(self.conv1(x)))
        # conv2 레이어에 relu 후 maxpool. 56 x 56 x 64
        x = self.pool(torch.relu(self.conv2(x)))
        # conv3 레이어에 relu 후 maxpool. 28 x 28 x 128
        x = self.pool(torch.relu(self.conv3(x)))
        # conv4 레이어에 relu 후 maxpool. 14 x 14 x 128
        x = self.pool(torch.relu(self.conv4(x)))

        # 이미지 펴기
        x = x.view(-1, 128 * 14 * 14) #2차원으로 만들때 첫번째 차원은 알아서 계산 (batch의 개수 유지해주기)
        # dropout 적용
        x = self.dropout(x)
        # fc 레이어에 삽입 후 relu
        x = torch.relu(self.fc1(x))
        # dropout 적용
        x = self.dropout(x)

        x = self.fc2(x)
        return x

In [8]:
model = Net().to(device)

In [9]:
def count_parameters(model):
    total_param = 0
    for name, param in model.named_parameters():
        if param.requires_grad:
            num_param = np.prod(param.size())
            if param.dim() > 1:
                print(name, ':', ' x '.join(str(x) for x in list(param.size())[::-1]), '=', num_param)
            else:
                print(name, ':', num_param)
                print('-' * 40)
            total_param += num_param
    print('total:', total_param)

In [10]:
count_parameters(model)

conv1.weight : 3 x 3 x 3 x 32 = 864
conv1.bias : 32
----------------------------------------
conv2.weight : 3 x 3 x 32 x 64 = 18432
conv2.bias : 64
----------------------------------------
conv3.weight : 3 x 3 x 64 x 128 = 73728
conv3.bias : 128
----------------------------------------
conv4.weight : 3 x 3 x 128 x 128 = 147456
conv4.bias : 128
----------------------------------------
fc1.weight : 25088 x 512 = 12845056
fc1.bias : 512
----------------------------------------
fc2.weight : 512 x 2 = 1024
fc2.bias : 2
----------------------------------------
total: 13087426


In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01) #adam 으로 학습이 안됨 stochastic gradident descent
loss_fn = nn.CrossEntropyLoss().to(device)

n_epochs = 20
loss = 0.0
#save_loss = 1.0
total_batch = len(trainloader)

model.train() #pytorch에서 모델의 학습 모드를 설정하는 메소드. 모델의 파라미터 업데이트 및 그래디언트 계산 가능 (보통 모델 학습 전에 붙여줌)
#1. dropout, batch normalization와 같이 학습 과정에서 사용되는 regularization 기법 동작.
#2. 그래디언트 계산을 가능하게 함. 학습 모드에서는 모델의 파라미터 업데이트를 위해 전파(backpropagation)를 수행하고, 그래디언트를 계산
#3. 학습 모드에서는 모델의 파라미터를 최적화(optimizer) 알고리즘에 따라 업데이트
for epoch in tqdm(range(n_epochs)):
    train_loss = 0.0
    for data, label in trainloader:
        optimizer.zero_grad()
        out = model(data.to(device))
        loss = loss_fn(out, label.to(device))
        loss.backward()
        optimizer.step()

        train_loss += loss / total_batch
        
    train_loss = train_loss/len(trainloader.sampler)
    # training set, validation set 로스율 출력
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch+1, float(train_loss)))

  0%|          | 0/20 [00:00<?, ?it/s]

Epoch: 1 	Training Loss: 0.000034
Epoch: 2 	Training Loss: 0.000032
Epoch: 3 	Training Loss: 0.000031
Epoch: 4 	Training Loss: 0.000029
Epoch: 5 	Training Loss: 0.000028
Epoch: 6 	Training Loss: 0.000026
Epoch: 7 	Training Loss: 0.000025
Epoch: 8 	Training Loss: 0.000024
Epoch: 9 	Training Loss: 0.000022
Epoch: 10 	Training Loss: 0.000022
Epoch: 11 	Training Loss: 0.000020
Epoch: 12 	Training Loss: 0.000019
Epoch: 13 	Training Loss: 0.000018
Epoch: 14 	Training Loss: 0.000017
Epoch: 15 	Training Loss: 0.000016
Epoch: 16 	Training Loss: 0.000015
Epoch: 17 	Training Loss: 0.000014
Epoch: 18 	Training Loss: 0.000012
Epoch: 19 	Training Loss: 0.000012
Epoch: 20 	Training Loss: 0.000011


In [21]:
total_batch #len trainloader

1000

In [22]:
len(testloader)

250

In [25]:
for data, label in tqdm(testloader):
    print(data)

  0%|          | 0/250 [00:00<?, ?it/s]

tensor([[[[-1.9809, -1.9809, -1.9809,  ..., -0.5082, -0.4911, -0.4226],
          [-1.9809, -1.9809, -1.9809,  ..., -0.5424, -0.4911, -0.4397],
          [-1.9809, -1.9809, -1.9809,  ..., -0.5424, -0.4911, -0.4397],
          ...,
          [-1.1589, -1.1247, -1.1589,  ..., -0.7993, -0.7993, -0.7479],
          [-1.1932, -1.1760, -1.1760,  ..., -0.8335, -0.8164, -0.7822],
          [-1.2103, -1.1760, -1.1418,  ..., -0.8678, -0.8335, -0.8507]],

         [[-1.8957, -1.8957, -1.8957,  ..., -0.6877, -0.6352, -0.5651],
          [-1.8957, -1.8957, -1.8957,  ..., -0.7052, -0.6352, -0.5826],
          [-1.8957, -1.8957, -1.8957,  ..., -0.7052, -0.6352, -0.5826],
          ...,
          [-1.9307, -1.8782, -1.9307,  ..., -1.8782, -1.8782, -1.8256],
          [-1.8957, -1.8782, -1.9132,  ..., -1.8957, -1.8957, -1.8431],
          [-1.8081, -1.8256, -1.8431,  ..., -1.9132, -1.8782, -1.8606]],

         [[-1.6650, -1.6650, -1.6650,  ..., -0.9853, -0.9504, -0.8633],
          [-1.6650, -1.6650, -

KeyboardInterrupt: 

In [33]:
classes= ['cat', 'dog']
test_loss = 0.0
correct = 0.0
class_correct = [0.0, 0.0]
class_total = [0.0, 0.0]

model.eval() #학습할 때만 dropout, batchnorm 등의 기능을 비활성화 해줘서 추론 할 때의 모드로 작동하도록 조정해 주는 역할
with torch.no_grad(): #test할 때 기울기 필요 없는 autograd engine을 비활 시켜서 메모리를 줄여주고 연산속도를 증가시켜줌
    for data, label in tqdm(testloader): #4차원, 1차원
        output = model(data.to(device)) #2차원
        #_, pred_index = torch.max(output, dim=1) #dim 1 기준으로 최대값 뽑고, 그 index 반환, _,pred_index: 앞에 것(max 값) 무시하고 뒤만 만듦 (max index) #https://paintycode.tistory.com/26
        pred_index=torch.max(output, dim=1)[1]
        #pred와 데이터를 비교한다
        correct_tensor = pred_index.eq(label.to(device).view_as(pred_index)) #view_as() 함수는 target 텐서를 view_as() 함수 안에 들어가는 인수(pred_index)의 모양대로 다시 정렬한다.
        # correct_tensor를 numpy로 바꾼 뒤 gpu 계산 또는 cpu 계산
        correct = np.squeeze(correct_tensor.numpy()) \
                        if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy()) #항목값이 1인 차원 날림
    # 몇 개 맞췄나 계산
        for i in range(batch_size): # 배치 사이즈로
            target = label[i] # tensor(0) or tensor(1) #T/F
            class_correct[target.item()] += correct[i]
            class_total[target.item()] += 1


for i in range(2):
    # 각 클래스 별 확률 출력
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

# 최종 확률 출력
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))









  0%|          | 0/250 [00:00<?, ?it/s]

[ True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True False]
[ True  True False  True  True  True  True  True  True  True  True  True
  True  True  True  True  True False  True  True]
[ True  True False False  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True False]
[ True False  True  True  True  True  True  True False False  True  True
  True  True  True  True  True  True  True False]
[ True  True  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True False  True]
[ True  True  True  True  True  True  True  True  True  True False  True
  True False  True False  True  True  True False]
[False False  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True  True  True  True]
[ True  True  True False  True  True  True False  True  True  True  True
  True False  True  True  True  True  True  True]
[ True  True  Tr

KeyboardInterrupt: 